# Description

According to the settings specified below, this notebook:
 1. reads all the data from one source (GTEx, recount2, etc) according to the gene selection method (`GENE_SELECTION_STRATEGY`),
 2. runs a quick performance test using the correlation coefficient specified (`CORRELATION_METHOD`), and
 3. computes the correlation matrix across all the genes using the correlation coefficient specified.

# Modules

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

# Paths

In [2]:
TOP_N_GENES = "all"

In [ ]:
DATA_DIR = Path("/mnt/data/proj_data/ccc-gpu/gene_expr/data/gtex_v8")
INPUT_DIR = DATA_DIR / "similarity_matrices/" / TOP_N_GENES
display(INPUT_DIR)

assert INPUT_DIR.exists()

PosixPath('/mnt/data/proj_data/ccc-gpu/gene_expr/data/gtex_v8/similarity_matrices/all')

# Data loading

In [4]:
cpu_res = pd.read_pickle(
    INPUT_DIR / f"gtex_v8_data_whole_blood-var_pc_log2-ccc-{TOP_N_GENES}.pkl"
)
gpu_res = pd.read_pickle(
    INPUT_DIR / f"gtex_v8_data_whole_blood-var_pc_log2-ccc_gpu-{TOP_N_GENES}.pkl"
)

In [5]:
cpu_res.shape

(56200, 56200)

In [ ]:
cpu_res.head()

gene_ens_id,ENSG00000169429.10,ENSG00000135245.9,ENSG00000163631.16,ENSG00000277632.1,ENSG00000239839.6,ENSG00000186652.9,ENSG00000129824.15,ENSG00000152463.14,ENSG00000123689.5,ENSG00000012223.12,...,ENSG00000199224.1,ENSG00000271798.1,ENSG00000229686.1,ENSG00000234273.1,ENSG00000271368.1,ENSG00000278424.1,ENSG00000199024.1,ENSG00000278281.1,ENSG00000232661.1,ENSG00000206780.1
gene_ens_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000169429.10,1.000000,0.780181,0.401717,0.789583,0.139359,0.524274,0.048667,0.365544,0.799041,0.343411,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000135245.9,0.780181,1.000000,0.429092,0.780181,0.159947,0.516615,0.042631,0.334136,0.752314,0.355974,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000163631.16,0.401717,0.429092,1.000000,0.408476,0.127681,0.401717,0.020316,0.232700,0.381776,0.261762,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000277632.1,0.789583,0.780181,0.408476,1.000000,0.151543,0.547588,0.055712,0.346530,0.761547,0.381776,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000239839.6,0.139359,0.159947,0.127681,0.151543,1.000000,0.272752,0.008065,0.100699,0.135411,0.318958,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
gpu_res.shape

(56200, 56200)

In [8]:
gpu_res.head()

gene_ens_id,ENSG00000169429.10,ENSG00000135245.9,ENSG00000163631.16,ENSG00000277632.1,ENSG00000239839.6,ENSG00000186652.9,ENSG00000129824.15,ENSG00000152463.14,ENSG00000123689.5,ENSG00000012223.12,...,ENSG00000199224.1,ENSG00000271798.1,ENSG00000229686.1,ENSG00000234273.1,ENSG00000271368.1,ENSG00000278424.1,ENSG00000199024.1,ENSG00000278281.1,ENSG00000232661.1,ENSG00000206780.1
gene_ens_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000169429.10,1.000000,0.780181,0.401717,0.789583,0.139359,0.524274,0.048667,0.365544,0.799041,0.343411,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000135245.9,0.780181,1.000000,0.429092,0.780181,0.159947,0.516615,0.042631,0.334136,0.752314,0.355974,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000163631.16,0.401717,0.429092,1.000000,0.408477,0.127681,0.401717,0.020316,0.232700,0.381776,0.261762,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000277632.1,0.789583,0.780181,0.408477,1.000000,0.151543,0.547588,0.055712,0.346530,0.761547,0.381776,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000239839.6,0.139359,0.159947,0.127681,0.151543,1.000000,0.272752,0.008065,0.100699,0.135411,0.318958,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
gpu_res["ENSG00000234741.7"]

gene_ens_id
ENSG00000169429.10    0.135411
ENSG00000135245.9     0.114192
ENSG00000163631.16    0.079873
ENSG00000277632.1     0.123901
ENSG00000239839.6     0.040029
                        ...   
ENSG00000278424.1     0.000000
ENSG00000199024.1     0.000000
ENSG00000278281.1     0.000000
ENSG00000232661.1     0.000000
ENSG00000206780.1     0.000000
Name: ENSG00000234741.7, Length: 56200, dtype: float32

In [10]:
cpu_res["ENSG00000234741.7"]

gene_ens_id
ENSG00000169429.10    0.135411
ENSG00000135245.9     0.114192
ENSG00000163631.16    0.079873
ENSG00000277632.1     0.123901
ENSG00000239839.6     0.040029
                        ...   
ENSG00000278424.1          NaN
ENSG00000199024.1          NaN
ENSG00000278281.1          NaN
ENSG00000232661.1          NaN
ENSG00000206780.1          NaN
Name: ENSG00000234741.7, Length: 56200, dtype: float64

In [11]:
# Assert the results are close
gpu_res = gpu_res.astype(np.float64)  # convert gpu_res to float64
pd.testing.assert_frame_equal(cpu_res, gpu_res, atol=1e-7)  # default atol is 1e-8

AssertionError: DataFrame.iloc[:, 0] (column name="ENSG00000169429.10") are different

DataFrame.iloc[:, 0] (column name="ENSG00000169429.10") values are different (5.16548 %)
[index]: [ENSG00000169429.10, ENSG00000135245.9, ENSG00000163631.16, ENSG00000277632.1, ENSG00000239839.6, ENSG00000186652.9, ENSG00000129824.15, ENSG00000152463.14, ENSG00000123689.5, ENSG00000012223.12, ENSG00000177606.6, ENSG00000158050.4, ENSG00000100448.3, ENSG00000196415.9, ENSG00000156575.2, ENSG00000163958.13, ENSG00000204388.6, ENSG00000172232.9, ENSG00000249138.1, ENSG00000197561.6, ENSG00000164821.4, ENSG00000115590.13, ENSG00000275302.1, ENSG00000203814.6, ENSG00000204936.9, ENSG00000005381.7, ENSG00000102837.6, ENSG00000090104.11, ENSG00000121053.5, ENSG00000168209.4, ENSG00000181631.6, ENSG00000220267.1, ENSG00000197582.5, ENSG00000147872.9, ENSG00000110245.11, ENSG00000240247.7, ENSG00000211896.7, ENSG00000158874.11, ENSG00000148346.11, ENSG00000146122.16, ENSG00000155659.14, ENSG00000164120.13, EN